In [ ]:
import holoviews as hv
from holoviews.operation.datashader import aggregate, shade, datashade, dynspread

hv.extension('bokeh')

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from lib.models import get_linear_model, get_mca_mod
from lib.util import mat_to_xarray
from lib.hvops import percent

from xnoah.data_matrix import unstack_cat


def output_to_xr(y, coords):
    res =  xr.DataArray(y, coords)
    return unstack_cat(res, "features").unstack("samples")

def resid_from_data(get_model, data):
    mod = get_model(data)
    mod.fit(*data['train'])
    
    x, y = data['train']
    
    # get prediction
    y_pred = mod.predict(x)

    # unstack
    y, y_pred= [output_to_xr(a, y.coords) for a in [y, y_pred]]
    

    return y, y_pred


In [ ]:
# load data
data = joblib.load("../data/ml/ngaqua/data.pkl")

resid_lm = resid_from_data(get_linear_model, data)
resid_mcr = resid_from_data(get_mca_mod, data)

data = xr.concat(resid_lm + (resid_mcr[1],), dim=pd.Index(["true", "lm", "mcr"], name="model"))

In [ ]:
%%opts Curve[invert_axes=True] {+framewise}
lay = hv.Dataset(data.Q1c.isel(x=0,y=8))\
.to.curve("z", dynamic=True)\
.overlay("model")
lay

In [ ]:
%%output dpi=150
%%opts Image[colorbar=True, width=400](cmap='viridis') 
%%opts GridSpace[shared_xaxis=True, shared_yaxis=True, xaxis=None]
hv.Dataset(data.Q1c.isel(y=8,x=120))\
.to.image(["time", "z"])\
.add_dimension("nil", 0, "")\
.grid()

We can see that MCR performs much worse than the lienar regression. And that the raw linear regression actually performs pretty well. Even though the linear response function of linear regression looks ugly it does yield a good prediction.

In [ ]:
df = data.Q1c.to_dataset(dim="model").to_dataframe()

In [ ]:
tab = hv.Table(df.reset_index())
scat  = (tab.to.scatter(kdims=["lm"], vdims=["true"], groupby="z"))

The linear model tends to underestimate the heating for large heating rates

In [ ]:
%%opts Curve(color="black")
datashade(percent(scat), cmap="blue")\
.redim.range(true=(0,1), lm=(0,1)) \
.redim.unit(lm="%", true="%")\
* hv.Curve(((0,1),(0,1)))